In [1]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr

df = pd.read_csv('stock_list/stock_list_20240126.csv', encoding='euc-kr')

### 기본필터링
1. 시장구분: 'KONEX' 제외
  - ['KOSDAQ' 'KOSPI' 'KONEX' 'KOSDAQ GLOBAL']
2. 증권구분: '주권'만 포함
  - ['주권' '부동산투자회사' '주식예탁증권' '외국주권' '사회간접자본투융자회사' '투자회사']
3. 소속부: '투자주의환기종목(소속부없음)', '외국기업(소속부없음)', '관리종목(소속부없음)', 'SPAC(소속부없음)' 제외
  - ['중견기업부' '우량기업부' nan '벤처기업부' '투자주의환기종목(소속부없음)' '외국기업(소속부없음)' '일반기업부' '기술성장기업부' '관리종목(소속부없음)' 'SPAC(소속부없음)']
4. 주식종류: '보통주'만 포함
  - ['보통주' '구형우선주' '신형우선주' '종류주권']

In [2]:
# 시장구분: 'KONEX' 제외
df = df[df['시장구분'] != 'KONEX']
# 증권구분: '주권'만 포함
df = df[df['증권구분'] == '주권']
# '투자주의환기종목(소속부없음)', '외국기업(소속부없음)', '관리종목(소속부없음)', 'SPAC(소속부없음)'
df = df[df['소속부'] != '투자주의환기종목(소속부없음)']
df = df[df['소속부'] != '외국기업(소속부없음)']
df = df[df['소속부'] != '관리종목(소속부없음)']
df = df[df['소속부'] != 'SPAC(소속부없음)']
# 주식종류: '보통주'만 포함
df = df[df['주식종류'] == '보통주']
df = df.reset_index(drop=True)
df

,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수
0,KR7098120009,098120,(주)마이크로컨텍솔루션,마이크로컨텍솔,"Micro Contact Solution Co.,Ltd.",2008/09/23,KOSDAQ,주권,중견기업부,보통주,500,8312766
1,KR7009520008,009520,(주)포스코엠텍,포스코엠텍,"POSCO M-TECH CO.,LTD.",1997/11/10,KOSDAQ,주권,우량기업부,보통주,500,41642703
2,KR7095570008,095570,AJ네트웍스보통주,AJ네트웍스,"AJ Networks Co.,Ltd.",2015/08/21,KOSPI,주권,NaN,보통주,1000,45252759
3,KR7006840003,006840,AK홀딩스보통주,AK홀딩스,"AK Holdings, Inc.",1999/08/11,KOSPI,주권,NaN,보통주,5000,13247561
4,KR7282330000,282330,BGF리테일보통주,BGF리테일,BGF Retail,2017/12/08,KOSPI,주권,NaN,보통주,1000,17283906
...,...,...,...,...,...,...,...,...,...,...,...,...
2325,KR7189980006,189980,흥국에프엔비,흥국에프엔비,"HYUNGKUK F&B Co., Ltd",2015/08/07,KOSDAQ,주권,우량기업부,보통주,100,40137827
2326,KR7000540005,000540,흥국화재해상보험보통주,흥국화재,HeungkukFire&MarineInsurance,1974/12/05,KOSPI,주권,NaN,보통주,5000,64242645
2327,KR7003280005,003280,흥아해운보통주,흥아해운,HeungaShipping,1976/06/29,KOSPI,주권,NaN,보통주,500,240424899
2328,KR7037440005,037440,희림종합건축사사무소,희림,Heerim Architects & Planners,2000/02/03,KOSDAQ,주권,우량기업부,보통주,500,13922475


In [3]:
print('시장구분', df['시장구분'].unique())
print('증권구분', df['증권구분'].unique())
print('소속부', df['소속부'].unique())
print('주식종류', df['주식종류'].unique())

시장구분 ['KOSDAQ' 'KOSPI' 'KOSDAQ GLOBAL']
증권구분 ['주권']
소속부 ['중견기업부' '우량기업부' nan '벤처기업부' '기술성장기업부']
주식종류 ['보통주']


In [4]:
import FinanceDataReader

df_krx = FinanceDataReader.StockListing("KRX")
df_krx_marcap = df_krx[['Code', 'Marcap']]
df_krx_marcap

,Code,Marcap
0,005930,444151821720000
1,000660,98280319275000
2,373220,88335000000000
3,207940,56939200000000
4,005935,49784645350000
...,...,...
2784,288490,2830395528
2785,217320,2787750000
2786,308700,2468000000
2787,245450,2369940000


In [5]:
df_full = pd.merge(df, df_krx_marcap, how='left', left_on='단축코드', right_on='Code')[['단축코드', '한글 종목약명', '상장일', '시장구분', '소속부', '상장주식수', 'Marcap']]
df_full

,단축코드,한글 종목약명,상장일,시장구분,소속부,상장주식수,Marcap
0,098120,마이크로컨텍솔,2008/09/23,KOSDAQ,중견기업부,8312766,99171298380
1,009520,포스코엠텍,1997/11/10,KOSDAQ,우량기업부,41642703,1003589142300
2,095570,AJ네트웍스,2015/08/21,KOSPI,NaN,45252759,215403132840
3,006840,AK홀딩스,1999/08/11,KOSPI,NaN,13247561,212093451610
4,282330,BGF리테일,2017/12/08,KOSPI,NaN,17283906,2499252807600
...,...,...,...,...,...,...,...
2325,189980,흥국에프엔비,2015/08/07,KOSDAQ,우량기업부,40137827,88503908535
2326,000540,흥국화재,1974/12/05,KOSPI,NaN,64242645,199473412725
2327,003280,흥아해운,1976/06/29,KOSPI,NaN,240424899,861923262915
2328,037440,희림,2000/02/03,KOSDAQ,우량기업부,13922475,97457325000


In [6]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2330 entries, 0 to 2329
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   단축코드     2330 non-null   object
 1   한글 종목약명  2330 non-null   object
 2   상장일      2330 non-null   object
 3   시장구분     2330 non-null   object
 4   소속부      1520 non-null   object
 5   상장주식수    2330 non-null   int64 
 6   Marcap   2330 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 127.5+ KB


In [7]:
df_full.columns = ['stock_code', 'corp_name', 'ipo_date', 'marker_type', 'dept', 'stock_num', 'market_cap']
df_full

,stock_code,corp_name,ipo_date,marker_type,dept,stock_num,market_cap
0,098120,마이크로컨텍솔,2008/09/23,KOSDAQ,중견기업부,8312766,99171298380
1,009520,포스코엠텍,1997/11/10,KOSDAQ,우량기업부,41642703,1003589142300
2,095570,AJ네트웍스,2015/08/21,KOSPI,NaN,45252759,215403132840
3,006840,AK홀딩스,1999/08/11,KOSPI,NaN,13247561,212093451610
4,282330,BGF리테일,2017/12/08,KOSPI,NaN,17283906,2499252807600
...,...,...,...,...,...,...,...
2325,189980,흥국에프엔비,2015/08/07,KOSDAQ,우량기업부,40137827,88503908535
2326,000540,흥국화재,1974/12/05,KOSPI,NaN,64242645,199473412725
2327,003280,흥아해운,1976/06/29,KOSPI,NaN,240424899,861923262915
2328,037440,희림,2000/02/03,KOSDAQ,우량기업부,13922475,97457325000


In [8]:
df_full = df_full.sort_values('market_cap', ascending=False).reset_index(drop=True)
df_full

,stock_code,corp_name,ipo_date,marker_type,dept,stock_num,market_cap
0,005930,삼성전자,1975/06/11,KOSPI,NaN,5969782550,444151821720000
1,000660,SK하이닉스,1996/12/26,KOSPI,NaN,728002365,98280319275000
2,373220,LG에너지솔루션,2022/01/27,KOSPI,NaN,234000000,88335000000000
3,207940,삼성바이오로직스,2016/11/10,KOSPI,NaN,71174000,56939200000000
4,005380,현대차,1974/06/28,KOSPI,NaN,211531506,41375562573600
...,...,...,...,...,...,...,...
2325,030790,비케이탑스,2010/11/02,KOSPI,NaN,21248145,19208323080
2326,071950,코아스,2005/08/04,KOSPI,NaN,30907210,15824491520
2327,078860,아이오케이,2006/06/27,KOSDAQ,중견기업부,4806323,15788771055
2328,008500,일정실업,1994/08/12,KOSPI,NaN,1200000,15480000000


In [9]:
df_full['ipo_date'] = pd.to_datetime(df_full['ipo_date'])
df_full

,stock_code,corp_name,ipo_date,marker_type,dept,stock_num,market_cap
0,005930,삼성전자,1975-06-11,KOSPI,NaN,5969782550,444151821720000
1,000660,SK하이닉스,1996-12-26,KOSPI,NaN,728002365,98280319275000
2,373220,LG에너지솔루션,2022-01-27,KOSPI,NaN,234000000,88335000000000
3,207940,삼성바이오로직스,2016-11-10,KOSPI,NaN,71174000,56939200000000
4,005380,현대차,1974-06-28,KOSPI,NaN,211531506,41375562573600
...,...,...,...,...,...,...,...
2325,030790,비케이탑스,2010-11-02,KOSPI,NaN,21248145,19208323080
2326,071950,코아스,2005-08-04,KOSPI,NaN,30907210,15824491520
2327,078860,아이오케이,2006-06-27,KOSDAQ,중견기업부,4806323,15788771055
2328,008500,일정실업,1994-08-12,KOSPI,NaN,1200000,15480000000


In [10]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2330 entries, 0 to 2329
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   stock_code   2330 non-null   object        
 1   corp_name    2330 non-null   object        
 2   ipo_date     2330 non-null   datetime64[ns]
 3   marker_type  2330 non-null   object        
 4   dept         1520 non-null   object        
 5   stock_num    2330 non-null   int64         
 6   market_cap   2330 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 127.5+ KB
